In [172]:
import numpy as np
import pandas as pd
import time
import datetime
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [161]:
features = pd.read_csv('./features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [162]:
# Selecting data for the first 5 minuts of game

features_train = features.loc[:,:'dire_first_ward_time']

features_train.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [269]:
features_train

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1450265551,1,47,4,1706,1198,17,0,1,8,...,0,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0
114403,1450277704,0,43,4,1793,1416,17,0,1,5,...,0,-5.0,NaN,-82.0,NaN,4,3,2,0,-17.0
114404,1450291848,1,98,4,1399,540,1,0,0,5,...,2,-32.0,249.0,-70.0,NaN,1,1,3,1,-15.0


In [163]:
# Features with missed data detecting:

col_name = features_train.columns
features_missed_data = []

for name in col_name:
    if features_train[name].count() < 97230: # features_train lenght = 97230, as evaluated above
        features_missed_data.append(name)

features_missed_data

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

In [192]:
# Replacing missed data with 0:
features_train_filled = features_train.fillna(0)

# Checking the result
features_train_filled['dire_first_ward_time'].count()

97230

In [165]:
X_train = features_train_filled.to_numpy()
X_train

array([[ 1.43019877e+09,  7.00000000e+00,  1.10000000e+01, ...,
         2.00000000e+00,  2.00000000e+00, -5.20000000e+01],
       [ 1.43022034e+09,  0.00000000e+00,  4.20000000e+01, ...,
         3.00000000e+00,  1.00000000e+00, -5.00000000e+00],
       [ 1.43022708e+09,  7.00000000e+00,  3.30000000e+01, ...,
         3.00000000e+00,  1.00000000e+00,  1.30000000e+01],
       ...,
       [ 1.45029185e+09,  1.00000000e+00,  9.80000000e+01, ...,
         3.00000000e+00,  1.00000000e+00, -1.50000000e+01],
       [ 1.45029299e+09,  1.00000000e+00,  1.00000000e+02, ...,
         3.00000000e+00,  1.00000000e+00, -4.20000000e+01],
       [ 1.45031337e+09,  7.00000000e+00,  5.00000000e+01, ...,
         2.00000000e+00,  0.00000000e+00, -3.90000000e+01]])

In [166]:
y_train = features['radiant_win']
y_train

match_id
0         1
1         1
2         0
3         0
4         0
         ..
114402    0
114403    1
114404    0
114405    0
114406    1
Name: radiant_win, Length: 97230, dtype: int64

In [156]:
# Gradient Boosting Classifier with standard max_depth evaluation:

n_estimators = [10, 20, 30, 40, 50]
scores = []
kf = KFold(shuffle=True, random_state=241)

for n in n_estimators:
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n, random_state=241)
    cvs = cross_val_score(clf, X_train, y_train, cv=kf, scoring='roc_auc')
    time = datetime.datetime.now() - start_time
    clf.fit(X_train, y_train)
    auc_roc = roc_auc_score(y_train, clf.decision_function(X_train)) # checking the auc-roc without cross-validation
    scores.append([n, cvs.mean(), auc_roc, time])
    
scores = pd.DataFrame(scores, columns = ['n_estimators', 'cvs.mean', 'auc-roc', 'time'])
scores

,n_estimators,cvs.mean,auc-roc,time
0,10,0.664388,0.672452,00:00:59.232925
1,20,0.682854,0.691589,00:02:02.335663
2,30,0.689496,0.700209,00:02:50.411444
3,40,0.694131,0.705587,00:03:52.761067
4,50,0.697455,0.710178,00:04:51.191606


In [157]:
scores.loc[lambda df: df['n_estimators'] == 30, :]

,n_estimators,cvs.mean,auc-roc,time
2,30,0.689496,0.700209,00:02:50.411444


In [158]:
# Gradient Boosting Classifier with max_depth=2 evaluation:

n_estimators = [10, 20, 30, 40, 50]
scores = []
kf = KFold(shuffle=True, random_state=241)

for n in n_estimators:
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n, random_state=241, max_depth=2)
    cvs = cross_val_score(clf, X_train, y_train, cv=kf, scoring='roc_auc')
    time = datetime.datetime.now() - start_time
    scores.append([n, cvs.mean(), time])
    
scores = pd.DataFrame(scores, columns = ['n_estimators', 'auc-roc', 'time'])
scores

,n_estimators,auc-roc,time
0,10,0.650843,00:00:40.778691
1,20,0.673530,00:01:20.891474
2,30,0.683069,00:01:58.709649
3,40,0.687910,00:02:33.646468
4,50,0.691551,00:03:15.913362


In [159]:
scores.loc[lambda df: df['n_estimators'] == 30, :]

,n_estimators,auc-roc,time
2,30,0.683069,00:01:58.709649


In [169]:
# Logistic regression evaluation with all features without scaling:
scores = []
kf = KFold(shuffle=True, random_state=241)
C = 0.00001

while C <= 100000:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(random_state=241, C=C)
    cvs = cross_val_score(clf, X_train, y_train, cv=kf, scoring='roc_auc')
    time = datetime.datetime.now() - start_time
    scores.append([C, cvs.mean(), time])
    C = C*10
    
scores = pd.DataFrame(scores, columns = ['C', 'auc-roc', 'time'])
scores

,C,auc-roc,time
0,0.00001,0.513456,00:00:01.461803
1,0.00010,0.513456,00:00:00.925439
2,0.00100,0.513456,00:00:00.937243
3,0.01000,0.513456,00:00:00.920622
4,0.10000,0.513456,00:00:00.920842
5,1.00000,0.513456,00:00:00.919864
6,10.00000,0.513456,00:00:00.920046
7,100.00000,0.513456,00:00:00.924565
8,1000.00000,0.513456,00:00:00.935406
9,10000.00000,0.513456,00:00:00.974800


In [170]:
# Scaling X_train for Logistic regression evaluation:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# Logistic regression evaluation with all features with scaling:
scores = []
kf = KFold(shuffle=True, random_state=241)
C = 0.00001

while C <= 100000:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(random_state=241, C=C)
    cvs = cross_val_score(clf, X_train_scaled, y_train, cv=kf, scoring='roc_auc')
    time = datetime.datetime.now() - start_time
    scores.append([C, cvs.mean(), time])
    C = C*10
    
scores = pd.DataFrame(scores, columns = ['C', 'auc-roc', 'time'])
scores

,C,auc-roc,time
0,0.00001,0.695115,00:00:01.280429
1,0.00010,0.711245,00:00:01.429568
2,0.00100,0.716179,00:00:02.491321
3,0.01000,0.716342,00:00:03.479517
4,0.10000,0.716310,00:00:03.844233
5,1.00000,0.716307,00:00:03.843197
6,10.00000,0.716306,00:00:03.530179
7,100.00000,0.716307,00:00:03.777960
8,1000.00000,0.716307,00:00:03.937021
9,10000.00000,0.716307,00:00:03.569537


In [173]:
# Logistic regression evaluation using GS with all features without scaling:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
kf = KFold(shuffle=True, random_state=241)
clf = LogisticRegression(random_state=241)
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=kf)
gs.fit(X_train, y_train)
C = gs.best_params_['C']
print(C)

1e-05


In [174]:
# Logistic regression evaluation using GS with all features with scaling:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
kf = KFold(shuffle=True, random_state=241)
clf = LogisticRegression(random_state=241)
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=kf)
gs.fit(X_train_scaled, y_train)
C = gs.best_params_['C']
print(C)

0.01


In [175]:
# Results matches with results evaluated via loops, best C = 0.01, better to use scaled X

In [188]:
# Finding best score:
scores.loc[lambda df: df['auc-roc'] == scores['auc-roc'].max(), :]

,C,auc-roc,time
3,0.01,0.716342,00:00:03.479517


In [196]:
# Deleting categorical features:
X_train_clean = features_train_filled.drop(['lobby_type','r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                                            'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
X_train_clean.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [197]:
# Scaling X_train_clean for Logistic regression evaluation:
scaler = StandardScaler()
X_train_clean_scaled = scaler.fit_transform(X_train_clean)

In [198]:
# Logistic regression evaluation with droped features with scaling:
scores = []
kf = KFold(shuffle=True, random_state=241)
C = 0.00001

while C <= 100000:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(random_state=241, C=C)
    cvs = cross_val_score(clf, X_train_clean_scaled, y_train, cv=kf, scoring='roc_auc')
    time = datetime.datetime.now() - start_time
    scores.append([C, cvs.mean(), time])
    C = C*10
    
scores = pd.DataFrame(scores, columns = ['C', 'auc-roc', 'time'])
scores

,C,auc-roc,time
0,0.00001,0.695051,00:00:01.262680
1,0.00010,0.711242,00:00:01.308300
2,0.00100,0.716235,00:00:02.593009
3,0.01000,0.716400,00:00:03.323209
4,0.10000,0.716375,00:00:03.343963
5,1.00000,0.716371,00:00:03.839791
6,10.00000,0.716371,00:00:03.648632
7,100.00000,0.716371,00:00:03.632692
8,1000.00000,0.716371,00:00:03.377016
9,10000.00000,0.716371,00:00:03.517972


In [199]:
# Finding best score:
scores.loc[lambda df: df['auc-roc'] == scores['auc-roc'].max(), :]

,C,auc-roc,time
3,0.01,0.7164,00:00:03.323209


In [231]:
# Checking the unique heroes:

In [256]:
h = []
for i in range(5):
    for hero in features_train_filled['r%d_hero' % (i+1)].unique():
        h.append([hero, (i+1)])
h = pd.DataFrame(h, columns = ['heroes', 'i'])
print(len(h.heroes)/5)

108.0


In [257]:
h = []
for i in range(5):
    for hero in features_train_filled['d%d_hero' % (i+1)].unique():
        h.append([hero, (i+1)])
h = pd.DataFrame(h, columns = ['heroes', 'i'])
print(len(h.heroes)/5)

108.0


In [ ]:
# Number of unique heroes in train dataset is 108

In [236]:
# Convert categorical variable for heroes into indicator variables:
X_pick = np.zeros((features_train_filled.shape[0], 113)) 

for i, match_id in enumerate(features_train_filled.index):
    for p in range(5):
        X_pick[i, features_train_filled.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features_train_filled.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [237]:
X_pick

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  0.]])

In [258]:
X_train = np.concatenate([X_train_clean_scaled, X_pick], axis = 1)
X_train

array([[-2.54436416,  1.40080818,  1.52597175, ...,  0.        ,
         0.        ,  0.        ],
       [-2.54045236,  0.50131354, -0.08013929, ...,  0.        ,
         0.        ,  0.        ],
       [-2.53923104,  0.50131354,  0.1510701 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.09874571,  0.50131354,  0.29226667, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.09895204, -0.39818111, -0.17368203, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.1026479 , -0.39818111, -0.31840851, ...,  0.        ,
        -1.        ,  0.        ]])

In [259]:
# Logistic regression evaluation with converted categorical features:
scores = []
kf = KFold(shuffle=True, random_state=241)
C = 0.00001

while C <= 100000:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(random_state=241, C=C)
    cvs = cross_val_score(clf, X_train, y_train, cv=kf, scoring='roc_auc')
    time = datetime.datetime.now() - start_time
    scores.append([C, cvs.mean(), time])
    C = C*10
    
scores = pd.DataFrame(scores, columns = ['C', 'auc-roc', 'time'])
scores

/Users/anastasia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/anastasia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/module

/Users/anastasia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/anastasia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/module

/Users/anastasia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/anastasia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/module

,C,auc-roc,time
0,0.00001,0.699127,00:00:02.060940
1,0.00010,0.724991,00:00:02.168122
2,0.00100,0.746294,00:00:04.811699
3,0.01000,0.751736,00:00:08.596459
4,0.10000,0.751937,00:00:11.352773
5,1.00000,0.751919,00:00:12.113292
6,10.00000,0.751918,00:00:12.017674
7,100.00000,0.751918,00:00:11.242370
8,1000.00000,0.751919,00:00:11.260705
9,10000.00000,0.751918,00:00:11.646268


In [300]:
# Finding best score:
scores.loc[lambda df: df['auc-roc'] == scores['auc-roc'].max(), :]

,C,auc-roc,time
4,0.1,0.751937,00:00:11.352773


In [ ]:
# Best model is Logistic Regression with C=0.1 and in X we should take converted heroes features

In [262]:
# test sample preparation:
data_test = pd.read_csv('./features_test.csv', index_col='match_id')
data_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [271]:
data_test

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114369,1450212780,7,11,5,2054,1941,27,0,1,8,...,1,8.0,253.0,-87.0,NaN,4,3,2,1,-33.0
114377,1450222875,1,3,3,748,605,1,0,0,12,...,1,-1.0,133.0,-85.0,184.0,2,3,4,1,-18.0
114378,1450223593,1,85,2,575,499,0,0,0,8,...,0,20.0,133.0,-88.0,239.0,4,4,4,0,-36.0


In [272]:
# Features with missed data detecting:

col_name = data_test.columns
test_missed_data = []

for name in col_name:
    if data_test[name].count() < 17177:
        test_missed_data.append(name)

test_missed_data

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

In [273]:
# Replacing missed data with 0:
data_test_filled = data_test.fillna(0)

# Checking the result
data_test_filled['dire_first_ward_time'].count()

17177

In [276]:
# Deleting categorical features:
X_test_clean = data_test_filled.drop(['lobby_type','r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                                            'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
X_test_clean.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,4,1103,1089,8,0,1,9,3,1183,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,2,556,570,1,0,0,9,4,1194,...,2,-29.0,168.0,-54.0,0.0,3,2,2,1,16.0
10,1430301774,2,751,808,1,0,0,13,2,421,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,3,708,903,1,1,1,11,2,672,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,4,1259,661,4,0,0,9,5,1703,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [279]:
# Scaling X_train_clean for Logistic regression evaluation:
scaler = StandardScaler()
X_test_clean_scaled = scaler.fit_transform(X_test_clean)

In [280]:
# Convert categorical variable for heroes into indicator variables:
X_pick = np.zeros((data_test_filled.shape[0], 113)) 

for i, match_id in enumerate(data_test_filled.index):
    for p in range(5):
        X_pick[i, data_test_filled.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data_test_filled.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [281]:
X_test = np.concatenate([X_test_clean_scaled, X_pick], axis = 1)
X_test

array([[-2.5148749 ,  0.47474566, -0.23774298, ...,  0.        ,
         0.        ,  0.        ],
       [-2.51389501, -1.36563036, -1.20791332, ...,  0.        ,
         0.        ,  0.        ],
       [-2.5123772 , -1.36563036, -0.86205735, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 1.08005567, -1.36563036, -1.17421453, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.08387463,  0.47474566,  1.0765097 , ...,  0.        ,
        -1.        ,  0.        ],
       [ 1.08579655,  0.47474566, -0.0390975 , ...,  0.        ,
         1.        ,  0.        ]])

In [282]:
clf = LogisticRegression(random_state=241, C=0.1)
clf.fit(X_train, y_train)

/Users/anastasia/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=241, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [285]:
predictions = clf.predict_proba(X_test)

In [284]:
clf.classes_

array([0, 1])

In [ ]:
# class 1 for Radiant win, so probability in second column

In [296]:
result = pd.DataFrame(predictions, columns = [['Dire', 'Radiant']])
result.head()

,Dire,Radiant
0,0.175398,0.824602
1,0.242720,0.757280
2,0.812486,0.187514
3,0.137193,0.862807
4,0.760071,0.239929


In [297]:
result.max() # finding max probability

Dire       0.991416
Radiant    0.996460
dtype: float64

In [298]:
result.min() # finding min probability

Dire       0.003540
Radiant    0.008584
dtype: float64